In [87]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
from statsmodels.stats.multitest import multipletests

In [88]:

#csv1
all_mean_arr=-0.019077723390066942
all_std_arr=0.006128439002438663
mean_dict_different_race={'ACB': -0.014011151662888553,
 'ASW': -0.012693412503283,
 'BEB': -0.0038608773522230016,
 'CDX': -0.000870975201046489,
 'CEU': -0.002626550761399808,
 'CHB': -0.000840617782511786,
 'CHS': -0.0016030210248706835,
 'CLM': -0.006497201413442441,
 'ESN': -0.012407888605124987,
 'FIN': -0.0013888123211964092,
 'GBR': -0.0016421960521901954,
 'GIH': -0.0029655045615120147,
 'GWD': -0.013383054011719266,
 'IBS': -0.0036274568161741316,
 'ITU': -0.003697573961398361,
 'JPT': -0.0004422965646677968,
 'KHV': -0.0014824412579579188,
 'LWK': -0.013214685221562708,
 'MSL': -0.012902619299438558,
 'MXL': -0.004064132377929528,
 'PEL': -0.0032051883341473507,
 'PJL': -0.004661546996727815,
 'PUR': -0.008252100717713692,
 'STU': -0.003961436794723387,
 'TSI': -0.002358177545570631,
 'YRI': -0.012978184004079407}
std_dict_different_race={'ACB': 0.007679014864719199,
 'ASW': 0.008041527656473924,
 'BEB': 0.008061181937627181,
 'CDX': 0.008574772155886829,
 'CEU': 0.007805712926154353,
 'CHB': 0.00858396197930932,
 'CHS': 0.008210661522162982,
 'CLM': 0.007855389049385354,
 'ESN': 0.007528169549036397,
 'FIN': 0.008233373830804186,
 'GBR': 0.008280482586380011,
 'GIH': 0.008318428884896655,
 'GWD': 0.00741808675422558,
 'IBS': 0.007946168606506706,
 'ITU': 0.007944085877932789,
 'JPT': 0.008520600208015637,
 'KHV': 0.008457471872578115,
 'LWK': 0.00775427047797345,
 'MSL': 0.00788405338804666,
 'MXL': 0.008244211463289736,
 'PEL': 0.008220014590454098,
 'PJL': 0.007728364636381623,
 'PUR': 0.007789028974214059,
 'STU': 0.007989081367465312,
 'TSI': 0.008171487042712589,
 'YRI': 0.007574843363674686}

In [90]:

mhc_race_positions_dict={'ACB': {'mhc': [[32589647, 32805573]]},
'All':{'mhc':[[32453110, 32577355], [32589647, 32716541]]},
 'ASW': {'mhc': [[29753369, 29913914],
   [29939668, 30120966],
   [32627859, 32843772]]},
 'BEB': {},
 'CDX': {'mhc': [[31295439, 31432528]]},
 'CEU': {'mhc': [[32627859, 32776791]]},
 'CHB': {},
 'CHS': {'mhc': [[32589647, 32716541]]},
 'CLM': {'mhc': [[32453110, 32577355], [32589647, 32732048]]},
 'ESN': {'mhc': [[29720403, 29896285], [32644320, 32843772]]},
 'FIN': {},
 'GBR': {'mhc': [[32453110, 32577355]]},
 'GIH': {'mhc': [[32589647, 32698571]]},
 'GWD': {'mhc': [[29720403, 29913914],
   [29939668, 30085606],
   [32627859, 32732048]]},
 'IBS': {'mhc': [[32453110, 32577355], [32589647, 32716541]]},
 'ITU': {'mhc': [[32423532, 32554290]]},
 'JPT': {'mhc': [[32589647, 32732048]]},
 'KHV': {'mhc': [[32453110, 32577355], [32589647, 32716541]]},
 'LWK': {'mhc': [[32627859, 32805573]]},
 'MSL': {'mhc': [[29720403, 29913914],
   [31317765, 31528792],
   [32554291, 32776791]]},
 'MXL': {'mhc': [[32589647, 32716541]]},
 'PEL': {'mhc': [[32473902, 32616414]]},
 'PJL': {},
 'PUR': {'mhc': [[32589647, 32683157]]},
 'STU': {'mhc': [[32453110, 32577355], [32589647, 32716541]]},
 'TSI': {'mhc': [[32589647, 32698571]]},
 'YRI': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [32589647, 32882258]]}}

In [92]:
split_race_region={'ACB': {'split_chr2/xau': [[89762579, 89852967]],
  'split_chr5/xaa': [[676694, 846549]],
  'split_chr7/xan': [[56467013, 56713542]],
  'split_chr11/xaw': [[103206633, 103451621]],
  'split_chr14/xaa': [[19806333, 19976578]],
  'split_chr21/xaa': [[10605321, 10733724]]},
  'All':{'split_chr14_xaa': [[19806333, 19976578]],
 'split_chr7_xbc': [[124810550, 125070920]],
 'split_chr1_xak': [[45762038, 46100296]],
 'split_chr1_xbk': [[188243362, 188513984]]},
 'ASW': {'split_chr2/xau': [[89762579, 89852967]],
  'split_chr5/xaa': [[676694, 846549]],
  'split_chr9/xak': [[42796452, 42969270]],
  'split_chr10/xaj': [[38491573, 38687800], [38725237, 38832458]],
  'split_chr11/xaf': [[23272931, 23495433]],
  'split_chr13/xaa': [[18347994, 18534139]],
  'split_chr14/xak': [[66346187, 66752333]],
  'split_chr21/xaa': [[10605321, 10733724]]},
 'BEB': {'split_chr1/xbx': [[245939605, 246162003]],
  'split_chr2/xan': [[56405225, 56644106]],
  'split_chr3/xav': [[98046342, 98273693]],
  'split_chr3/xbf': [[145365797, 145612467]],
  'split_chr4/xaz': [[115247002, 115485969]],
  'split_chr13/xai': [[55102733, 55443491]],
  'split_chr14/xaa': [[19806333, 19976578]]},
 'CDX': {'split_chr1/xay': [[114816228, 115116610]],
  'split_chr2/xbd': [[130125757, 130339027]],
  'split_chr4/xba': [[119293026, 119539168]],
  'split_chr6/xao': [[64160994, 64523871]],
  'split_chr7/xaf': [[19734421, 19953255]],
  'split_chr9/xau': [[105652970, 105876427]],
  'split_chr10/xan': [[55664607, 55894887]],
  'split_chr12/xbc': [[131390693, 131619328]],
  'split_chr13/xai': [[55102733, 55354096]],
  'split_chr17/xag': [[31068982, 31384157]],
  'split_chr18/xan': [[65996302, 66233011]]},
 'CEU': {'split_chr1/xbe': [[158399387, 158641706]],
  'split_chr6/xac': [[8413699, 8649798]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr11/xaf': [[24934206, 25144712]],
  'split_chr12/xac': [[10962097, 11205540]],
  'split_chr17/xaf': [[21588491, 21823237]],
  'split_chr17/xaj': [[45620566, 46418024]]},
 'CHB': {'split_chr2/xbd': [[130125757, 130339027]],
  'split_chr4/xba': [[119293026, 119587470]],
  'split_chr4/xaz': [[115247002, 115485969]],
  'split_chr6/xao': [[64202650, 64523871]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr7/xaf': [[19775051, 19953255]],
  'split_chr10/xar': [[73060579, 73337762]],
  'split_chr11/xav': [[98072700, 98330585]],
  'split_chr22/xaa': [[16175432, 16345129]]},
 'CHS': {'split_chr3/xav': [[98046342, 98245327]],
  'split_chr4/xba': [[119293026, 119539168]],
  'split_chr5/xaa': [[676694, 846549]],
  'split_chr6/xao': [[64202650, 64523871]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr8/xaz': [[109058766, 109345554]],
  'split_chr14/xat': [[105509278, 105695533]]},
 'CLM': {'split_chr1/xbd': [[152162701, 152471006]],
  'split_chr3/xav': [[95059735, 95322404], [98046342, 98273693]],
  'split_chr3/xbj': [[163862679, 164224214]],
  'split_chr4/xba': [[119293026, 119587470]],
  'split_chr6/xab': [[7345120, 7558507]],
  'split_chr6/xar': [[76169005, 76460170]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr12/xan': [[61013177, 61213757]],
  'split_chr14/xaa': [[19806333, 19976578]],
  'split_chr14/xat': [[106520496, 106674883]]},
 'ESN': {'split_chr2/xau': [[89581761, 89852967]],
  'split_chr5/xaa': [[676694, 846549]],
  'split_chr5/xai': [[34311870, 34562837]],
  'split_chr6/xbf': [[145443018, 145762353]],
  'split_chr7/xap': [[67044394, 67267582]],
  'split_chr14/xan': [[80889782, 81152442]],
  'split_chr21/xaa': [[10605321, 10733724]]},
 'FIN': {'split_chr3/xbj': [[163862679, 164147845]],
  'split_chr4/xaw': [[97892649, 98142264]],
  'split_chr4/xbh': [[151414350, 151728888]],
  'split_chr6/xat': [[85153819, 85387984]],
  'split_chr6/xbf': [[145485121, 145762353]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr8/xaz': [[109010721, 109345554]],
  'split_chr9/xav': [[112161299, 112408537]],
  'split_chr11/xbb': [[124171937, 124450010]],
  'split_chr18/xac': [[12017790, 12244103]]},
 'GBR': {'split_chr1/xak': [[45628366, 46147211]],
  'split_chr2/xan': [[56405225, 56644106]],
  'split_chr2/xas': [[78325225, 78577282]],
  'split_chr3/xbf': [[145365797, 145612467]],
  'split_chr3/xbi': [[158122703, 158435520]],
  'split_chr4/xba': [[119243365, 119539168]],
  'split_chr6/xac': [[8413699, 8649798]],
  'split_chr7/xbc': [[124760998, 125070920]],
  'split_chr11/xbb': [[124171937, 124450010]],
  'split_chr12/xac': [[10962097, 11205540]],
  'split_chr17/xaf': [[21654230, 21823237]],
  'split_chr17/xaj': [[45587933, 46418024]]},
 'GIH': {'split_chr3/xav': [[98046342, 98276757]],
  'split_chr4/xbd': [[132580391, 132816794]],
  'split_chr6/xac': [[8413699, 8649798]],
  'split_chr11/xah': [[34650776, 34905225]],
  'split_chr14/xaa': [[19806333, 20015421]],
  'split_chr18/xac': [[12017790, 12244103]]},
 'GWD': {'split_chr1/xak': [[45803471, 46100296]],
  'split_chr2/xai': [[38264369, 38464894]],
  'split_chr2/xau': [[89762579, 89852967]],
  'split_chr5/xaa': [[676694, 846549]],
  'split_chr13/xaa': [[18347994, 18534139]],
  'split_chr21/xaa': [[10511846, 10763989]]},
 'IBS': {'split_chr3/xbf': [[145365797, 145612467]],
  'split_chr6/xbf': [[145485121, 145762353]],
  'split_chr12/xac': [[10962097, 11205540]],
  'split_chr12/xas': [[83579779, 83843102]],
  'split_chr17/xaf': [[21619971, 21823237]],
  'split_chr17/xaj': [[45836662, 46418024]],
  'split_chr18/xac': [[12017790, 12244103]],
  'split_chr22/xaa': [[16131708, 16345129]]},
 'ITU': {'split_chr5/xas': [[84862293, 85266031]],
  'split_chr10/xan': [[55664607, 55894887]],
  'split_chr12/xac': [[10962097, 11162840]],
  'split_chr14/xaa': [[19806333, 20047342]],
  'split_chr17/xak': [[52575391, 52889013]]},
 'JPT': {'split_chr3/xav': [[98046342, 98245327]],
  'split_chr5/xbc': [[131688028, 132052794]],
  'split_chr5/xaa': [[676694, 825167]],
  'split_chr13/xaa': [[18347994, 18534139]],
  'split_chr13/xap': [[85638733, 85884384]],
  'split_chr14/xad': [[31681845, 31909184]],
  'split_chr22/xaa': [[16175432, 16345129]]},
 'KHV': {'split_chr2/xbq': [[192771748, 193054217]],
  'split_chr3/xav': [[98046342, 98245327]],
  'split_chr3/xbj': [[163946649, 164185047]],
  'split_chr4/xba': [[119293026, 119539168]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr14/xak': [[66298204, 66793088]]},
 'LWK': {'split_chr2/xai': [[38235897, 38464894]],
  'split_chr2/xau': [[89762579, 89852967]],
  'split_chr5/xaa': [[676694, 846549]],
  'split_chr13/xaa': [[18347994, 18534139]],
  'split_chr14/xak': [[66346187, 66752333]],
  'split_chr21/xaa': [[10605321, 10733724]]},
 'MSL': {'split_chr1/xak': [[45803471, 46100296]],
  'split_chr2/xau': [[89762579, 89852967]],
  'split_chr5/xay': [[113365150, 113639918]],
  'split_chr7/xaf': [[19775051, 19953255]],
  'split_chr7/xap': [[67044394, 67267582]],
  'split_chr9/xak': [[42796452, 42969270]],
  'split_chr11/xaf': [[23272931, 23495433]],
  'split_chr11/xaw': [[103206633, 103451621]],
  'split_chr13/xaa': [[18347994, 18534139]],
  'split_chr14/xat': [[105569851, 105695533]],
  'split_chr21/xaa': [[10605321, 10733724]]},
 'MXL': {'split_chr3/xav': [[98046342, 98217489]],
  'split_chr3/xaq': [[74901608, 75192541]],
  'split_chr5/xad': [[11999758, 12263260]],
  'split_chr6/xao': [[64160994, 64404669]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr11/xaf': [[23200593, 23461651]],
  'split_chr12/xad': [[14846240, 15093187]],
  'split_chr12/xac': [[10962097, 11162840]],
  'split_chr12/xap': [[73338587, 73586502]],
  'split_chr12/xas': [[83579779, 83843102]],
  'split_chr17/xaf': [[21654230, 21823237]]},
 'PEL': {'split_chr1/xak': [[45628366, 46147211]],
  'split_chr2/xbo': [[186062200, 186331668]],
  'split_chr3/xav': [[98046342, 98217489]],
  'split_chr3/xba': [[120872663, 121116904]],
  'split_chr4/xaq': [[69225334, 69589122]],
  'split_chr6/xab': [[7345120, 7695065]],
  'split_chr8/xba': [[112137591, 112486758]],
  'split_chr10/xaq': [[68153912, 68481883]],
  'split_chr11/xaf': [[23168664, 23461651]],
  'split_chr11/xam': [[58630320, 58868016]],
  'split_chr12/xac': [[10962097, 11162840]],
  'split_chr12/xas': [[83579779, 83843102]],
  'split_chr14/xaa': [[19806333, 19976578]],
  'split_chr16/xab': [[5659954, 5828695]]},
 'PJL': {'split_chr12/xac': [[10962097, 11162840]],
  'split_chr14/xac': [[28519266, 28761965]],
  'split_chr14/xaa': [[19806333, 20078222]],
  'split_chr17/xak': [[52575391, 52861780]]},
 'PUR': {'split_chr3/xav': [[98003499, 98273693]],
  'split_chr3/xbj': [[163819938, 164063747]],
  'split_chr12/xac': [[10962097, 11205540]],
  'split_chr12/xan': [[61013177, 61213757]],
  'split_chr12/xas': [[83689105, 83928389]],
  'split_chr17/xaf': [[21654230, 21823237]]},
 'STU': {'split_chr3/xav': [[98046342, 98273693]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr13/xaa': [[18347994, 18534139]],
  'split_chr13/xai': [[55102733, 55443491]],
  'split_chr14/xaa': [[19806333, 19976578]],
  'split_chr16/xaj': [[48512974, 48799209]],
  'split_chr19/xam': [[51801039, 52098595]]},
 'TSI': {'split_chr4/xaw': [[97892649, 98189302]],
  'split_chr4/xba': [[119293026, 119539168]],
  'split_chr7/xag': [[23951217, 24204307]],
  'split_chr7/xbc': [[124810550, 125070920]],
  'split_chr9/xai': [[31957257, 32172669]],
  'split_chr12/xac': [[10962097, 11162840]],
  'split_chr14/xaa': [[19806333, 19976578]],
  'split_chr17/xaf': [[21654230, 21823237]],
  'split_chr17/xaj': [[45836662, 46544765]]},
 'YRI': {'split_chr2/xau': [[89762579, 89852967]],
  'split_chr5/xaa': [[676694, 846549]],
  'split_chr7/xan': [[56467013, 56713542]],
  'split_chr7/xap': [[67044394, 67267582]],
  'split_chr9/xak': [[42796452, 42969270]],
  'split_chr13/xaa': [[18347994, 18534139]],
  'split_chr16/xao': [[72288822, 72684953]],
  'split_chr21/xaa': [[10605321, 10733724]]}}

In [94]:
import re
def getinteger(s):
    match = re.search(r'chr(\d+)', s)

    if match:
        return match.group(1) 


In [95]:
df_mhc_all=pd.read_csv("../csvfiles/posmhc.csv")
df_mhc_race=pd.read_csv("../race/mhc.csv")


In [96]:
allrecords=[]
for race,values in mhc_race_positions_dict.items():
    if values!={}:
        if race=="All":
            for region in values["mhc"]:
                onerecord=[]
                onerecord+=["All",6,1]
                top_1000_P_list=df_mhc_all[(df_mhc_all["Unnamed: 0"]>=region[0])&(df_mhc_all["Unnamed: 0"]<=region[1])]["P"].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P, all_mean_arr, all_std_arr)),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)
        else:
            for region in values["mhc"]:
                onerecord=[]
                onerecord+=[race,6,1]
                top_1000_P_list=df_mhc_race[(df_mhc_race["Unnamed: 0"]>=region[0])&(df_mhc_race["Unnamed: 0"]<=region[1])][race].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P, mean_dict_different_race[race], std_dict_different_race[race])),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)


In [97]:
for race,values in split_race_region.items():
    if race=="All":
        for k, v in values.items():
            chr=getinteger(k.split("_")[1])
            filename="pos_prob_"+k+".csv"
            df=pd.read_csv("../csvfiles/"+filename)
            for region in v:
                onerecord=[]
                onerecord+=["All",chr,0]
                top_1000_P_list=df[(df["Unnamed: 0"]>=region[0])&(df["Unnamed: 0"]<=region[1])]["P"].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P, all_mean_arr, all_std_arr)),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)
    else:
        for k,v in values.items():
            chr=getinteger(k.split("/")[0])
            filename=k.split("/")[0]+"_"+k.split("/")[1]+".csv"
            df=pd.read_csv("../race/"+filename)
            for region in v:
                onerecord=[]
                onerecord+=[race,chr,0]
                top_1000_P_list=df[(df["Unnamed: 0"]>=region[0])&(df["Unnamed: 0"]<=region[1])][race].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P,mean_dict_different_race[race], std_dict_different_race[race])),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)

In [102]:
#csv1
columns=["Race","chr","isMHC","Average of P","Std of P","P-value","start","end","len"]

In [105]:
#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    return dataframe(dictforDF)





In [106]:
dfnew=generatedf(columns=columns,allrecords=allrecords)

In [107]:
dfnew

,Race,chr,isMHC,Average of P,Std of P,P-value,start,end,len
0,ACB,6,1,0.049944,0.138446,0.000000e+00,32589647,32805573,215926
1,All,6,1,0.006781,0.128870,1.224664e-05,32453110,32577355,124245
2,All,6,1,0.020229,0.107096,7.095100e-11,32589647,32716541,126894
3,ASW,6,1,0.024682,0.133795,1.677667e-06,29753369,29913914,160545
4,ASW,6,1,0.028085,0.130788,1.979418e-07,29939668,30120966,181298
...,...,...,...,...,...,...,...,...,...
246,YRI,7,0,0.014489,0.083202,1.438667e-04,67044394,67267582,223188
247,YRI,9,0,0.013714,0.092953,2.126580e-04,42796452,42969270,172818
248,YRI,13,0,0.028297,0.201186,2.533486e-08,18347994,18534139,186145
249,YRI,16,0,0.001372,0.051933,2.908446e-02,72288822,72684953,396131


In [108]:

pvalue_list=dfnew["P-value"].to_list()
pvalue_list

[0.0,
 1.2246640198e-05,
 7.0951e-11,
 1.677667317e-06,
 1.97941775e-07,
 5.9e-14,
 2.184838124e-06,
 3.116718113e-06,
 4.52545353e-07,
 6.25e-12,
 8.08e-12,
 5.104e-12,
 4.0138262e-08,
 4.061149e-09,
 8.1408642e-08,
 3.89e-13,
 2.079e-12,
 1.78057e-10,
 0.000639563652038,
 1.9783436e-08,
 4.38899202e-07,
 1.84318e-10,
 4.5667241746e-05,
 2e-15,
 1.03894239e-07,
 2e-15,
 7.162800705e-06,
 2.124679e-09,
 1.728e-12,
 1.957029e-09,
 0.003276692706815,
 9.47672677e-07,
 3e-13,
 9.5753e-11,
 2.44253e-09,
 5.810084e-09,
 2.02e-13,
 0.0,
 0.000537695911219,
 3.530501777e-06,
 0.00066017420478,
 2.6457656093e-05,
 0.007115550242639,
 1.0997967406e-05,
 0.00021279271811,
 0.013626083033611,
 0.001966959547561,
 0.0,
 0.0008519593589,
 3.8719575291e-05,
 0.017053425970364,
 0.015612652740083,
 0.003802114391157,
 2.3837276194e-05,
 0.000785596793686,
 0.01238731008931,
 0.012557849828423,
 0.000441584157919,
 8.657382943e-06,
 0.001798288718485,
 0.003262652281968,
 0.001193410768351,
 3.6703192

In [109]:
pvalue_list=dfnew["P-value"].to_list()
corr_pvals = multipletests(pvalue_list, method='fdr_bh')[1]
corr_pvals

array([0.00000000e+00, 4.26931485e-05, 5.93623367e-10, 7.13719486e-06,
       9.55449722e-07, 7.40450000e-13, 8.99007162e-06, 1.26176814e-05,
       2.02837292e-06, 5.81018519e-11, 7.24314286e-11, 4.92732308e-11,
       2.28970540e-07, 2.61371384e-08, 4.34756790e-07, 4.24517391e-12,
       2.08731600e-11, 1.35431233e-09, 1.15489552e-03, 1.21113230e-07,
       2.00297636e-06, 1.36070053e-09, 1.38102141e-04, 2.64210526e-14,
       5.32192939e-07, 2.64210526e-14, 2.72403481e-05, 1.48137341e-08,
       1.80720000e-11, 1.40346937e-08, 4.51895533e-03, 4.10113521e-06,
       3.42272727e-12, 7.75290419e-10, 1.65695954e-08, 3.64582771e-08,
       2.41438095e-12, 0.00000000e+00, 9.98752018e-04, 1.40659674e-05,
       1.17520373e-03, 8.40616668e-05, 8.23042908e-03, 3.88801383e-05,
       4.64443237e-04, 1.43703649e-02, 2.90415792e-03, 0.00000000e+00,
       1.47477103e-03, 1.18519676e-04, 1.73295948e-02, 1.59950034e-02,
       4.96670819e-03, 7.77033289e-05, 1.36933886e-03, 1.32872429e-02,
      

In [112]:
dfnew = dfnew.assign(corr_pvals=corr_pvals)


In [113]:
dfnew

,Race,chr,isMHC,Average of P,Std of P,P-value,start,end,len,corr_pvals
0,ACB,6,1,0.049944,0.138446,0.000000e+00,32589647,32805573,215926,0.000000e+00
1,All,6,1,0.006781,0.128870,1.224664e-05,32453110,32577355,124245,4.269315e-05
2,All,6,1,0.020229,0.107096,7.095100e-11,32589647,32716541,126894,5.936234e-10
3,ASW,6,1,0.024682,0.133795,1.677667e-06,29753369,29913914,160545,7.137195e-06
4,ASW,6,1,0.028085,0.130788,1.979418e-07,29939668,30120966,181298,9.554497e-07
...,...,...,...,...,...,...,...,...,...,...
246,YRI,7,0,0.014489,0.083202,1.438667e-04,67044394,67267582,223188,3.374817e-04
247,YRI,9,0,0.013714,0.092953,2.126580e-04,42796452,42969270,172818,4.644432e-04
248,YRI,13,0,0.028297,0.201186,2.533486e-08,18347994,18534139,186145,1.514060e-07
249,YRI,16,0,0.001372,0.051933,2.908446e-02,72288822,72684953,396131,2.908446e-02


In [ ]:
dfnew.to_csv("../python/csv1_0510.csv")